In [1]:
import main as coindb
import configparser
import util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

config = configparser.ConfigParser()
config.read('config.ini')
db_config = config['mysql']

db = coindb.DatabaseManagement(db_config)

In [2]:
df_close = db.get_df_variable()

In [3]:
def price_to_return(df_close):

    # return of crypto
    mat = ((df_close[1:].values / df_close[:-1].values) - 1)*100

    # screen by asset s.t. at least 800 day trading history
    condition_asset = (~np.isnan(mat)).sum(0)>800
    mat_temp = mat[:,condition_asset]
    df_temp = pd.DataFrame(mat_temp, columns=df_close.columns[condition_asset], index=df_close[1:].index)

    # screen by date 
    max_id = mat_temp.shape[0] - 1
    min_id = max_id - 800
    df_temp = pd.DataFrame(mat_temp[min_id:max_id,:], columns=df_temp.columns, index=df_temp.index[min_id:max_id])

    # final screen 
    condition = (np.isnan(df_temp).sum(0)<1)
    df_return = pd.DataFrame(df_temp.values[:,condition], columns=df_temp.columns[condition], index=df_temp.index)
    
    return df_return

In [4]:
price_to_return(df_close).to_csv('crypto_return_usd.csv')

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [5]:
import copy 

# btc return of crypto

mat_price = copy.deepcopy(df_close)

for coin in mat_price.columns:
    mat_price[coin] = mat_price[coin]/mat_price.BTC

price_to_return(mat_price).to_csv('crypto_return_btc.csv')
    

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
